In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from datasets import load_dataset
from sklearn.metrics import classification_report
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("iestynmullinor/roberta-reranker-fever")
model = AutoModelForSequenceClassification.from_pretrained("iestynmullinor/roberta-reranker-fever")



/home/iestyn/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("iestynmullinor/evidence_reranker_fever_balanced")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
test_data = dataset["test"]



test_data = test_data.map(lambda example: {'claim': example['claim'].lower() if isinstance(example['claim'], str) else example['claim'],
                                                          'evidence': example['evidence'].lower() if isinstance(example['evidence'], str) else example['evidence'],
                                                          'label': example['label']},batched=True)




In [4]:
import torch

inputs = tokenizer(test_data[0]["claim"], test_data[0]["evidence"], return_tensors="pt")

with torch.no_grad():

    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

print(predicted_class_id)

0


In [5]:
from tqdm import tqdm
predictions = []

for i in tqdm(range(len(test_data))):
    inputs = tokenizer(test_data[i]["claim"], test_data[i]["evidence"], return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()

    predictions.append(predicted_class_id)


100%|██████████| 13332/13332 [17:19<00:00, 12.82it/s]


In [6]:
gold_labels = [test_data[i]['label'] for i in range(len(test_data))]
print(classification_report(gold_labels, predictions))


              precision    recall  f1-score   support

           0       0.50      1.00      0.67      6649
           1       0.00      0.00      0.00      6683

    accuracy                           0.50     13332
   macro avg       0.25      0.50      0.33     13332
weighted avg       0.25      0.50      0.33     13332



/home/iestyn/miniconda3/envs/project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iestyn/miniconda3/envs/project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iestyn/miniconda3/envs/project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,